In [ ]:
import tensorflow as tf
from pyramda import compose, curry
import os
import numpy as np
import sys
from sklearn.model_selection import train_test_split
from visuazlizers.nb_graph_visualizer import show_graph, rename_nodes
from utils.curried_functions import tf_add, tf_cast, tf_multiply, filter_list
from loaders import image_batch_loader, load_model_pb
from siamese import compute_loss, get_anchor_positive_mask, get_negative_mask
from utils.metrics import cosine_distance

### Batch

In [ ]:
classes_to_labels = compose(
    list,
    range,
    len,
)

### Model

In [ ]:
from models.deep_sort_cnn.freeze_model import _preprocess, _network_factory

def create_deep_sort_conv_graph():
    input_var = tf.placeholder(tf.uint8, (None, 128, 64, 3), name="images")
    image_var = tf.map_fn(
        lambda x: _preprocess(x), tf.cast(input_var, tf.float32),
        back_prop=False
    )

    factory_fn = _network_factory()
    features, _ = factory_fn(image_var, reuse=None)
    features = tf.identity(features, name="features")
    
    return input_var, features

In [ ]:
inputs, outputs, graph_def = load_model_pb('./models/deep_sort_cnn/mars-small128.pb', input_name="images", output_name="features", graph_creator=create_deep_sort_conv_graph)

In [ ]:
tmp_def = rename_nodes(graph_def, lambda s:"/".join(s.split('_',1)))
show_graph(tmp_def)

### Training

In [ ]:
def train_siamese_model(
    session,
    model, 
    source_path, 
    dirs, 
    train_labels, 
    metric, 
    optimizer,
    batch_loader,
    margin=0.2, 
    num_per_class=5, 
    num_iter=1000, 
):
    """
    Trains a model
    
    Parameters:
    -----------
    - session: Tensorflow Session instance.
    - model: tuple
        Sould contain two tensors - one for moel input, another for model output.
    - source_path: string
        Path to model data.
    - dirs: [[string], [string]]
        Lists of training and validation directories.
    - train_labels: [int]
        Labels for classes used during training.
    - metric: Function
        Should take output tensor as a parameter and compute distance matrix between outputs.
    - optimizer: Tensorflow optimizer instance
    - batch_loader: Function
        Should take source_path, train_dirs, train_labels, num_per_class as parameters
        and return an iterator [samples, batch_lables]
    - margin: float
        Desired margin between negative and positive samples.
    - num_per_class: int
        Number of samples randomly chosen from each class
    - num_iter: int
        Number of iterations
    """
    
    train_dirs, val_dirs = dirs
    
    inputs, outputs = model
    labels = tf.placeholder(name='labels', shape=(len(train_labels) * num_per_class), dtype=tf.int8)
    anchor_positive_mask = get_anchor_positive_mask(labels)
    negetive_mask = get_negative_mask(labels)
    
    loss = compute_loss(
        model=(inputs, outputs), 
        metric=metric, 
        masks=(anchor_positive_mask, negetive_mask), 
        margin=margin,
    )
    
    train_step = optimizer.minimize(loss)
    session.run(tf.global_variables_initializer())
    
    for i in range(num_iter):
        samples, batch_lables = batch_loader(source_path, train_dirs, train_labels, num_per_class)
        batch_outputs, batch_loss, _ = session.run([outputs, loss, train_step], {
            inputs: samples,
            labels: batch_lables,
        })
        print(batch_loss)

In [ ]:
tf.reset_default_graph()

source_path = '../input/mars/bbox_train/'
dirs = compose(
    filter_list(['.DS_Store'], False),
    os.listdir,
)(source_path)

train_dirs, val_dirs = train_test_split(dirs, test_size=0.2)
train_labels = classes_to_labels(train_dirs)

inputs, outputs, _ = load_model_pb(
    './models/deep_sort_cnn/mars-small128.pb', 
    input_name="images", 
    output_name="features", 
    graph_creator=create_deep_sort_conv_graph,
)

session = tf.Session()

train_siamese_model(
    session=session,
    model=[inputs, outputs],
    source_path=source_path,
    dirs=(train_dirs[0:5], val_dirs),
    train_labels=train_labels[0:5],
    metric=cosine_distance,
    optimizer=tf.train.AdamOptimizer(learning_rate=0.00001),
    batch_loader=image_batch_loader((128, 64, 3)),
    num_iter=2,
)